In [3]:
import os
import json
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
np.bool = np.bool_

import mxnet as mx
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import OffsetSplitter
from gluonts.time_feature import TimeFeature
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.transform.sampler import ValidationSplitSampler

import torch
from lightning.pytorch.callbacks import Timer 

# 기본 라이브러리
import numpy as np
import pandas as pd
import torch
import os
import gc
import json
import pickle
from datetime import datetime
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm

# 수치 계산 관련
from scipy import linalg
from sklearn.preprocessing import StandardScaler
import h5py
import psutil

# 성능 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# GluonTS 관련
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from gluonts.mx import DeepAREstimator, MQRNNEstimator, TemporalFusionTransformerEstimator, DeepStateEstimator, DeepFactorEstimator, WaveNetEstimator, MQCNNEstimator, NBEATSEstimator, TransformerEstimator
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from gluonts.mx.distribution import GaussianOutput
from gluonts.mx import Trainer
from gluonts.evaluation import Evaluator
from gluonts.model.predictor import Predictor
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.dataset.loader import TrainDataLoader, InferenceDataLoader

import h5py
import gc
import psutil
from tqdm import tqdm
from datetime import datetime

import matplotlib.pyplot as plt

from gluonts.transform import UniformSplitSampler, InstanceSplitter
from gluonts.mx.batchify import batchify

from pathlib import Path


from utils import *

import warnings
warnings.filterwarnings("ignore", message="The mean prediction is not stored in the forecast data*")

np.random.seed(0)
mx.random.seed(0)

In [ ]:
def train_models(epochs, learning_rate, batch_size, save_dir='../result/test/'):
    os.makedirs(save_dir, exist_ok=True)

    level_estimators = {}
    level_preds = {}
    level_metrics = {}

    trainer = Trainer(         
        epochs=epochs,
        learning_rate=learning_rate,
        num_batches_per_epoch=100
    )
    
    for level_idx in range(1, 13):
        level_dir = os.path.join(save_dir, f'level_{level_idx}')
        os.makedirs(level_dir, exist_ok=True)

        highlight_print(f"Loading dataset for level {level_idx}")
        with open(os.path.join('../dataset/else', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
            dataset = pickle.load(f)
            train_dataset = dataset['train']
            test_dataset = dataset['test']

        highlight_print(f"Training model for level {level_idx}")

        if level_idx == 1:
            DeepAR = DeepAREstimator(
                freq="D",
                context_length=28,
                prediction_length=28,
                use_feat_dynamic_real=True,  
                use_feat_static_cat=False,    
                trainer=trainer,
            )
        else:
            DeepAR = DeepAREstimator(
                freq="D",
                context_length=28,
                prediction_length=28,
                use_feat_dynamic_real=True,  
                use_feat_static_cat=True,    
                cardinality=[len(train_dataset)],
                trainer=trainer,
            )
        MQRNN = MQRNNEstimator(
            freq="D",
            context_length=28,
            prediction_length=28,         
            trainer=trainer,
        )
        TFT = TemporalFusionTransformerEstimator(
            freq="D",
            context_length=28,
            prediction_length=28,
            dynamic_feature_dims = {
                "sales_mean": 1,
                "sales_std": 1,
                "sales_max": 1,
                "sales_min": 1,
                "sales_lag1": 1,
                "sales_lag7": 1,
                "sales_lag28": 1,
                "sales_rolling7_mean": 1,
                "sales_rolling28_mean": 1,
                "sales_trend": 1,
                "release_mean": 1,
                "out_of_stock_mean": 1,
                "sell_price_mean": 1,
                "sell_price_std": 1,
                "sell_price_max": 1,
                "sell_price_min": 1,
                "sell_price_diff": 1,
                "sell_price_trend": 1,
                "sell_price_in_store_mean": 1,
                "year_delta": 1,
                "quarter_sin": 1,
                "quarter_cos": 1,
                "month_sin": 1,
                "month_cos": 1,
                "day_sin": 1,
                "day_cos": 1,
                "weekday_sin": 1,
                "weekday_cos": 1,
                "event_count": 1,
            },
            dynamic_cardinalities={
                "snap_CA":2,
                "snap_TX":2,
                "snap_WI":2,
                "event_name_1_enc":31,
                "event_name_2_enc":31,
                "event_type_1_enc":5,
                "event_type_2_enc":5
            },
            static_cardinalities={"id":len(train_dataset)},
            trainer=trainer
        )
        if level_idx == 1:
            DeepState = DeepStateEstimator(
                freq="D",
                past_length=28,
                prediction_length=28,
                use_feat_dynamic_real=True,  
                use_feat_static_cat=False,
                cardinality=[len(train_dataset)],
                trainer=trainer,
            )
        else:
            DeepState = DeepStateEstimator(
                freq="D",
                past_length=28,
                prediction_length=28,
                use_feat_dynamic_real=True,  
                use_feat_static_cat=True,    
                cardinality=[len(train_dataset)],
                trainer=trainer,
            )
        DeepFactor = DeepFactorEstimator(
            freq="D",
            context_length=28,
            prediction_length=28,   
            cardinality=[len(train_dataset)],
            trainer=trainer,
        )
        WaveNet = WaveNetEstimator(
            freq="D",
            prediction_length=28,   
            cardinality=[len(train_dataset)],
            trainer=trainer,
        )
        MQCNN = MQCNNEstimator(
            freq="D",
            context_length=28,
            prediction_length=28,   
            use_feat_dynamic_real=True,  
            use_feat_static_cat=True,  
            cardinality=[len(train_dataset)],     
            trainer=trainer,
        )
        NBEATS = NBEATSEstimator(
            freq="D",
            context_length=28,
            prediction_length=28,   
            trainer=trainer,
        )
        Transformer = TransformerEstimator(
            freq="D",
            context_length=28,
            prediction_length=28,   
            use_feat_dynamic_real=True,  
            use_feat_static_cat=True,  
            cardinality=[len(train_dataset)],
            trainer=trainer,
        )
        predictor = MQRNN.train(train_dataset)

        highlight_print(f"Saving model for level {level_idx}")
        predictor.serialize(Path("/tmp/"))

        highlight_print(f"Making predictions for level {level_idx}")
        forecast_it, test_it = make_evaluation_predictions(
            dataset=test_dataset,
            predictor=predictor,
        )
        forecasts = list(forecast_it)
        tests = list(test_it)

        forecast_entry = forecasts[0]
        print(f"Number of sample paths: {forecast_entry.num_samples}")
        print(f"Dimension of samples: {forecast_entry.samples.shape}")
        print(f"Start date of the forecast window: {forecast_entry.start_date}")
        print(f"Frequency of the time series: {forecast_entry.freq}")
        print(f"Mean of the future window:\n {forecast_entry.mean}")
        print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")

        # # 예측 결과
        # level_pred = {}
        # for forecast in forecasts:
        #     id_preds = {quantile: forecast.quantile(quantile) for quantile in quantiles}
        #     level_pred[forecast.item_id] = id_preds

        # # 모델 평가
        # evaluator = Evaluator(quantiles=quantiles)
        # agg_metrics, item_metrics = evaluator(tests, forecasts)

        # # 결과 저장
        # level_estimators[level_idx+1] = estimator
        # level_preds[level_idx+1] = level_pred
        # level_metrics[level_idx+1] = agg_metrics

        # # 예측 결과 저장
        # with open(os.path.join(level_dir, 'predictions.pkl'), 'wb') as f:
        #     pickle.dump(level_preds[level_idx], f)
        
        # # 평가 지표 저장
        # metrics_df = pd.DataFrame(level_metrics[level_idx]).round(4)
        # metrics_df.to_csv(os.path.join(level_dir, 'metrics.csv'))
    
    # 하이퍼파라미터 저장
    # with open(os.path.join(save_dir, 'model_params.json'), 'w') as f:
    #     model_params = {
    #         'epochs': epochs,
    #         'learning_rate': lr,
    #         'batch_size': batch_size
    #     }
    #     json.dump(model_params, f, indent=4)


train_models(epochs=1, learning_rate=1e-3, batch_size=128, save_dir='../result/test/prac')

Loading dataset for level 1
Training model for level 1


 43%|████▎     | 43/100 [00:02<00:02, 19.09it/s, epoch=1/1, avg_epoch_loss=3.63e+3]
c:\Users\USER\anaconda3\envs\capstone\lib\site-packages\gluonts\mx\trainer\_base.py:484: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


UnboundLocalError: local variable 'epoch_loss' referenced before assignment

In [29]:
import os
import json
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
np.bool = np.bool_

import mxnet as mx
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import OffsetSplitter
from gluonts.time_feature import TimeFeature
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.transform.sampler import ValidationSplitSampler

import torch
from lightning.pytorch.callbacks import Timer 

# 기본 라이브러리
import numpy as np
import pandas as pd
import torch
import os
import gc
import json
import pickle
from datetime import datetime
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm

# 수치 계산 관련
from scipy import linalg
from sklearn.preprocessing import StandardScaler
import h5py
import psutil

# 성능 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# GluonTS 관련
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from gluonts.mx import DeepAREstimator, MQRNNEstimator, TemporalFusionTransformerEstimator, DeepStateEstimator, DeepFactorEstimator, WaveNetEstimator, MQCNNEstimator, NBEATSEstimator, TransformerEstimator
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from gluonts.mx.distribution import GaussianOutput
from gluonts.mx import Trainer
from gluonts.evaluation import Evaluator
from gluonts.model.predictor import Predictor
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.dataset.loader import TrainDataLoader, InferenceDataLoader

import h5py
import gc
import psutil
from tqdm import tqdm
from datetime import datetime

import matplotlib.pyplot as plt

from gluonts.transform import UniformSplitSampler, InstanceSplitter
from gluonts.mx.batchify import batchify

from pathlib import Path


from utils import *

import warnings
warnings.filterwarnings("ignore", message="The mean prediction is not stored in the forecast data*")

np.random.seed(0)
mx.random.seed(0)

In [30]:
epochs=3
learning_rate=1e-4
batch_size=128
save_dir='../result/test/prac'

os.makedirs(save_dir, exist_ok=True)

level_estimators = {}
level_preds = {}
level_metrics = {}

trainer = Trainer(         
    epochs=epochs,
    learning_rate=learning_rate,
    num_batches_per_epoch=100,
    clip_gradient=10.0 
)

for level_idx in range(10, 13):
    level_dir = os.path.join(save_dir, f'level_{level_idx}')
    os.makedirs(level_dir, exist_ok=True)

    highlight_print(f"Loading dataset for level {level_idx}")
    with open(os.path.join('../dataset/else', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
        dataset = pickle.load(f)
        train_dataset = dataset['train']
        test_dataset = dataset['test']

    highlight_print(f"Training model for level {level_idx}")
    if level_idx == 1:
        DeepAR = DeepAREstimator( # 10 x
            freq="D",
            context_length=28,
            prediction_length=28,
            use_feat_dynamic_real=True,  
            use_feat_static_cat=False,    
            trainer=trainer,
        )
    else:
        DeepAR = DeepAREstimator(
            freq="D",
            context_length=28,
            prediction_length=28,
            use_feat_dynamic_real=True,  
            use_feat_static_cat=True,    
            cardinality=[len(train_dataset)],
            trainer=trainer,
        )
    MQRNN = MQRNNEstimator(
        freq="D",
        context_length=28,
        prediction_length=28,         
        trainer=trainer,
    )
    TFT = TemporalFusionTransformerEstimator(
        freq="D",
        context_length=28,
        prediction_length=28,
        dynamic_feature_dims = {
            "sales_mean": 1,
            "sales_std": 1,
            "sales_max": 1,
            "sales_min": 1,
            "sales_lag1": 1,
            "sales_lag7": 1,
            "sales_lag28": 1,
            "sales_rolling7_mean": 1,
            "sales_rolling28_mean": 1,
            "sales_trend": 1,
            "release_mean": 1,
            "out_of_stock_mean": 1,
            "sell_price_mean": 1,
            "sell_price_std": 1,
            "sell_price_max": 1,
            "sell_price_min": 1,
            "sell_price_diff": 1,
            "sell_price_trend": 1,
            "sell_price_in_store_mean": 1,
            "year_delta": 1,
            "quarter_sin": 1,
            "quarter_cos": 1,
            "month_sin": 1,
            "month_cos": 1,
            "day_sin": 1,
            "day_cos": 1,
            "weekday_sin": 1,
            "weekday_cos": 1,
            "event_count": 1,
        },
        dynamic_cardinalities={
            "snap_CA":2,
            "snap_TX":2,
            "snap_WI":2,
            "event_name_1_enc":31,
            "event_name_2_enc":31,
            "event_type_1_enc":5,
            "event_type_2_enc":5
        },
        static_cardinalities={"id":len(train_dataset)},
        trainer=trainer
    )
    if level_idx == 1:
        DeepState = DeepStateEstimator(
            freq="D",
            past_length=28,
            prediction_length=28,
            use_feat_dynamic_real=True,  
            use_feat_static_cat=False,
            cardinality=[len(train_dataset)],
            trainer=trainer,
        )
    else:
        DeepState = DeepStateEstimator(
            freq="D",
            past_length=28,
            prediction_length=28,
            use_feat_dynamic_real=True,  
            use_feat_static_cat=True,    
            cardinality=[len(train_dataset)],
            trainer=trainer,
        )
    DeepFactor = DeepFactorEstimator(
        freq="D",
        context_length=28,
        prediction_length=28,   
        cardinality=[len(train_dataset)],
        trainer=trainer,
    )
    WaveNet = WaveNetEstimator(
        freq="D",
        prediction_length=28,   
        cardinality=[len(train_dataset)],
        trainer=trainer,
    )
    MQCNN = MQCNNEstimator(
        freq="D",
        context_length=28,
        prediction_length=28,   
        use_feat_dynamic_real=True,  
        use_feat_static_cat=True,  
        cardinality=[len(train_dataset)],     
        trainer=trainer,
    )
    NBEATS = NBEATSEstimator(
        freq="D",
        context_length=28,
        prediction_length=28,   
        trainer=trainer,
    )
    Transformer = TransformerEstimator(
        freq="D",
        context_length=28,
        prediction_length=28,   
        use_feat_dynamic_real=True,  
        use_feat_static_cat=True,  
        cardinality=[len(train_dataset)],
        trainer=trainer,
    )

    predictor = DeepAR.train(train_dataset)

    highlight_print(f"Making predictions for level {level_idx}")
    forecast_it, test_it = make_evaluation_predictions(
        dataset=test_dataset,
        predictor=predictor,
    )
    forecasts = list(forecast_it)
    tests = list(test_it)

    forecast_entry = forecasts[0]
    test_entry = tests[0]
    
    for idx in range(len(forecasts)):
        if idx == 3:
            break
        plt.figure(figsize=(10, 5))
        plt.plot(tests[idx][-100:].to_timestamp(), label="Actual")
        plt.plot(pd.Series(forecasts[idx].quantile(0.5), index=forecasts[idx].start_date.to_timestamp() + pd.to_timedelta(range(len(forecasts[idx].quantile(0.5))), unit='D')), label="Forecast")
        plt.title(f"Forecast for Time Series {idx + 1}")
        plt.legend()
        plt.show()


    break


Loading dataset for level 10
Training model for level 10


 65%|██████▌   | 65/100 [00:06<00:03,  9.45it/s, epoch=1/3, avg_epoch_loss=nan]
c:\Users\USER\anaconda3\envs\capstone\lib\site-packages\gluonts\mx\trainer\_base.py:484: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


UnboundLocalError: local variable 'epoch_loss' referenced before assignment